In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
np.set_printoptions(precision=10)

In [3]:
dev = qml.device('default.qubit', wires=2)

In [4]:
@qml.qnode(dev)
def circuit(params):
    qml.RY(params[0], wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [6]:
params = np.array([np.pi/4, np.pi/4])
print(circuit(params))

1.414213562373095


In [7]:
def my_finite_diff_grad(params, h=1.0e-7):
    gradient = np.zeros_like(params)
    
    for i in range(len(params)):
        params[i] += h
        gradient[i] += circuit(params)
        
        params[i] -= 2*h
        gradient[i] -= circuit(params)
        
        gradient[i] /= 2*h
        
        params[i] += h
        
    return gradient

In [11]:
@qml.qnode(dev, diff_method='finite-diff')
def circuit_finite_diff(params):
    qml.RY(params[0], wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [13]:
params = np.array([np.pi/4, np.pi/4], requires_grad=True)
print(my_finite_diff_grad(params))

[-0.7071067798 -0.707106782 ]


In [14]:
print(qml.grad(circuit_finite_diff)(params))

[-0.7071068153 -0.7071068175]


In [15]:
def my_param_shift_grad(params, s=np.pi/3):
    gradient = np.zeros_like(params)
    
    for i in range(len(params)):
        params[i] += s
        gradient[i] += circuit(params)
        
        params[i] -= 2*s
        gradient[i] -= circuit(params)
        
        gradient[i] /= 2*np.sin(s)
        
        params[i] += s
        
    return gradient

In [16]:
@qml.qnode(dev, diff_method='parameter-shift')
def circuit_parameter_shift(params):
    qml.RY(params[0], wires=0)
    qml.RX(params[1], wires=1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [17]:
print(my_param_shift_grad(params))

[-0.7071067812 -0.7071067812]


In [18]:
print(qml.grad(circuit_parameter_shift)(params))

[-0.7071067812 -0.7071067812]
